<a href="https://colab.research.google.com/github/Doffena/Large-Language-Models/blob/main/VERS%C4%B0ON2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings, sys, torch
warnings.filterwarnings("ignore", message=r".*HF_TOKEN.*", category=UserWarning)

print("Python:", sys.version)
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Torch: 2.9.0+cu126
CUDA available: True


In [ ]:
# ===== CLEAN + STABLE INSTALL (run once per fresh runtime) =====
!pip -q uninstall -y faiss-cpu faiss sentence-transformers gradio numpy \
  datasets transformers accelerate huggingface_hub peft bitsandbytes triton sentencepiece safetensors

# (Colab bazen "invalid distribution ~ip" kalıntısı bırakıyor; temizle)
!rm -rf /usr/local/lib/python3.12/dist-packages/~ip*

# FAISS için kritik: NumPy 2 değil, <2
!pip -q install "numpy<2.0"

# Senin sabitlediğin çekirdek kütüphaneler
!pip -q install datasets==2.21.0 transformers==4.47.1 accelerate==1.2.1 huggingface_hub==0.26.5 peft==0.13.2

# RAG için gerekenler (faiss-cpu + sentence-transformers + gradio)
!pip -q install faiss-cpu sentence-transformers gradio

import numpy as np
print("numpy version:", np.__version__)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtune 0.6.1 requires datasets, which is not installed.
torchtune 0.6.1 requires huggingface_hub[hf_transfer], which is not installed.
torchtune 0.6.1 requires safetensors, which is not installed.
torchtune 0.6.1 requires sentencepiece, which is not installed.
diffusers 0.36.0 requires huggingface-hub<2.0,>=0.34.0, which is not installed.
diffusers 0.36.0 requires safetensors>=0.3.1, which is not installed.
timm 1.0.22 requires huggingface_hub, which is not installed.
timm 1.0.22 requires safetensors, which is not installed.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompati

In [ ]:
import os
import re
import json
import hashlib
import unicodedata
import warnings
from dataclasses import dataclass
from typing import List, Optional, Dict, Any

from datasets import load_dataset
from difflib import SequenceMatcher

# (Opsiyonel) Colab'da HF_TOKEN uyarısını bastırmak istersen:
warnings.filterwarnings("ignore", message=r".*HF_TOKEN.*", category=UserWarning)

# -----------------------
# DATA SOURCE SETTINGS
# -----------------------
# Öncelik: dataset'i HF ID ile çek (daha stabil)
DATASET_ID = os.environ.get("DATASET_ID", "mertbozkurt/turkish-recipe")

# Fallback: senin kullandığın düz txt dosya yolu (HF hub file)
DATASET_TEXT_FILE = os.environ.get(
    "DATASET_TEXT_FILE",
    "hf://datasets/mertbozkurt/turkish-recipe/datav3.txt"
)

# Cache (Colab local disk)
CACHE_DIR = os.environ.get("CACHE_DIR", "/content/recipe_cache")
CACHE_PATH = os.path.join(CACHE_DIR, "recipes_v3.jsonl")

# Matching rules
FUZZY_THRESHOLD = int(os.environ.get("FUZZY_THRESHOLD", "90"))
ENABLE_FUZZY = False  # difflib fuzzy büyük listelerde pahalı
NOT_FOUND_MSG = "Bu tarif veri setimde bulunmuyor."
SUGGEST_PREFIX = "Bu tarif veri setimde bulunmuyor.\n\nBunu mu demek istediniz?"


# Training toggle: DO NOT start training unless you set this to True
RUN_TRAINING = False

# -----------------------
# PARSING REGEX
# -----------------------
_RECIPE_START_RE = re.compile(r"^(?P<title>.+?)\s+nas[ıi]l\s+yap[ıi]l[ıi]r\?\s*$", re.IGNORECASE)
_ING_HEADER_RE = re.compile(r"gerekli\s+malzemeler\s*:\s*$", re.IGNORECASE)
_STEPS_HEADER_RE = re.compile(r"yap[ıi]l[ıi][şs][ıi]\s*:\s*$", re.IGNORECASE)
_CATEGORY_RE = re.compile(r"^kategori\s*:\s*(?P<cat>.+)$", re.IGNORECASE)
_GARBAGE_PAT = re.compile(
    r"(Bu Tariflere de Göz At|Videolu Tarif|Aşağıdan Hemen İzleyebilirsiniz|Yemek Tarifleri|Püf Noktaları)",
    re.IGNORECASE
)

_TR_FOLD_MAP = str.maketrans({
    "ç": "c", "ğ": "g", "ı": "i", "İ": "i", "ö": "o", "ş": "s", "ü": "u",
    "Ç": "c", "Ğ": "g", "Ö": "o", "Ş": "s", "Ü": "u",
})

_QUERY_STRIP_PATTERNS = [
    r"\btarif(?:i|ini|im|imı|imi)?\b",
    r"\bnasil yapilir\b",
    r"\bnasıl yapılır\b",
    r"\bnedir\b",
    r"\bnasil\b",
    r"\bnasıl\b",
]


def normalize_text(s: str) -> str:
    s = (s or "").strip()
    if not s:
        return ""
    s = s.casefold()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = unicodedata.normalize("NFKC", s)
    s = s.translate(_TR_FOLD_MAP)
    s = re.sub(r"[^0-9a-z\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s


def extract_recipe_name(user_text: str) -> str:
    t = normalize_text(user_text)
    if not t:
        return ""
    for pat in _QUERY_STRIP_PATTERNS:
        t = re.sub(pat, " ", t, flags=re.IGNORECASE)
    t = re.sub(r"\s+", " ", t).strip()
    return t if len(t) >= 3 else ""


@dataclass(frozen=True)
class Recipe:
    title: str
    category: str
    ingredients: List[str]
    instructions: List[str]
    norm_title: str


def _fingerprint(r: Recipe) -> str:
    payload = {
        "t": r.norm_title,
        "c": (r.category or "").strip(),
        "i": [x.strip() for x in (r.ingredients or [])],
        "s": [x.strip() for x in (r.instructions or [])],
    }
    b = json.dumps(payload, ensure_ascii=False, separators=(",", ":")).encode("utf-8")
    return hashlib.sha1(b).hexdigest()


def _safe_read_cache() -> Optional[List[Recipe]]:
    if not os.path.exists(CACHE_PATH):
        return None
    recipes: List[Recipe] = []
    with open(CACHE_PATH, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
                recipes.append(
                    Recipe(
                        title=str(obj.get("title", "")),
                        category=str(obj.get("category", "")),
                        ingredients=list(obj.get("ingredients", []) or []),
                        instructions=list(obj.get("instructions", []) or []),
                        norm_title=str(obj.get("norm_title", "")),
                    )
                )
            except Exception:
                # bozuk satır varsa atla
                continue
    return recipes


def _write_cache(recipes: List[Recipe]) -> None:
    os.makedirs(CACHE_DIR, exist_ok=True)
    with open(CACHE_PATH, "w", encoding="utf-8") as f:
        for r in recipes:
            f.write(
                json.dumps(
                    {
                        "title": r.title,
                        "category": r.category,
                        "ingredients": r.ingredients,
                        "instructions": r.instructions,
                        "norm_title": r.norm_title,
                    },
                    ensure_ascii=False,
                )
                + "\n"
            )


def _load_lines_from_hf() -> List[str]:
    """
    Önce structured HF dataset'i dener.
    Eğer olmazsa, hf://.../datav3.txt text loader fallback yapar.
    """
    # 1) Try structured dataset
    try:
        ds = load_dataset(DATASET_ID, split="train")
        cols = set(ds.column_names)

        # Beklenen ihtimaller:
        # - "text" tek kolon (tam tarif)
        # - "title/materials/how-to/category" gibi kolonlar
        if "text" in cols:
            return [str(x or "").strip() for x in ds["text"]]
        if "content" in cols:
            return [str(x or "").strip() for x in ds["content"]]

        # Eğer alanlı dataset ise "satır" formatına biz çeviririz:
        # (Bu satırlar aşağıdaki parse logic ile değil, doğrudan recipe objeye de çevrilebilir,
        # ama senin mevcut parser'ın datav3 formatına göre yazıldığı için burada "line stream" üretmiyoruz.)
        # Bu durumda fallback'a geçiyoruz.
        raise ValueError(f"Dataset columns not supported for text-parse: {cols}")

    except Exception as e:
        # 2) Fallback to datav3.txt as text lines
        ds = load_dataset("text", data_files={"train": DATASET_TEXT_FILE}, split="train")
        return [str(row.get("text", "")).strip() for row in ds]


def load_recipes_v3(force_rebuild: bool = False) -> List[Recipe]:
    os.makedirs(CACHE_DIR, exist_ok=True)

    if not force_rebuild:
        cached = _safe_read_cache()
        if cached and len(cached) > 0:
            return cached

    lines = _load_lines_from_hf()

    recipes: List[Recipe] = []
    cur_title = ""
    cur_category = ""
    ingredients: List[str] = []
    steps: List[str] = []
    mode: Optional[str] = None

    def flush() -> None:
        nonlocal cur_title, cur_category, ingredients, steps, mode
        if cur_title:
            recipes.append(
                Recipe(
                    title=cur_title,
                    category=cur_category,
                    ingredients=ingredients[:],
                    instructions=steps[:],
                    norm_title=normalize_text(cur_title),
                )
            )
        cur_title = ""
        cur_category = ""
        ingredients = []
        steps = []
        mode = None

    for line in lines:
        line = (line or "").strip()

        if _GARBAGE_PAT.search(line):
            continue

        if not line:
            if cur_title:
                flush()
            continue

        m = _RECIPE_START_RE.match(line)
        if m:
            if cur_title:
                flush()
            cur_title = str(m.group("title")).strip()
            mode = None
            continue

        if not cur_title:
            continue

        mc = _CATEGORY_RE.match(line)
        if mc:
            cur_category = str(mc.group("cat")).strip()
            continue

        if _ING_HEADER_RE.search(line):
            mode = "ing"
            continue
        if _STEPS_HEADER_RE.search(line):
            mode = "steps"
            continue

        if mode == "ing":
            ingredients.append(line.lstrip("-•").strip())
        elif mode == "steps":
            steps.append(line.strip())
        else:
            steps.append(line.strip())

    if cur_title:
        flush()

    # dedupe exact duplicates
    seen = set()
    deduped: List[Recipe] = []
    for r in recipes:
        fp = _fingerprint(r)
        if fp in seen:
            continue
        seen.add(fp)
        deduped.append(r)
    recipes = deduped

    _write_cache(recipes)
    return recipes


def format_recipe_block(r: Recipe) -> str:
    parts: List[str] = []
    parts.append("Tarifin Adı:")
    parts.append(r.title.strip())
    parts.append("")
    parts.append("Malzemeler:")
    parts.extend([f"- {x}" for x in (r.ingredients or [])] or ["(bulunamadı)"])
    parts.append("")
    parts.append("Yapılışı:")
    if r.instructions:
        parts.extend([f"{i+1}. {x}" for i, x in enumerate(r.instructions)])
    else:
        parts.append("(bulunamadı)")
    return "\n".join(parts).strip()


def build_sft_example(r: Recipe) -> dict:
    prompt = f"[INST] {r.title} tarifi [/INST]"
    completion = format_recipe_block(r)
    return {"text": prompt + "\n" + completion}


class StrictMatcher:
    def __init__(self, recipes: List[Recipe]):
        self.recipes = recipes
        self.norm_titles = [r.norm_title for r in recipes]
        self.by_norm: Dict[str, List[int]] = {}
        for i, t in enumerate(self.norm_titles):
            self.by_norm.setdefault(t, []).append(i)

    def _pick_best_exact(self, idxs: List[int]) -> Optional[int]:
        if not idxs:
            return None
        if len(idxs) == 1:
            return idxs[0]

        def score(i: int):
            r = self.recipes[i]
            return (len(r.instructions or []), len(r.ingredients or []))
        return max(idxs, key=score)

    def match(self, user_text: str) -> Optional[Recipe]:
        candidate = extract_recipe_name(user_text)
        if not candidate:
            return None
        q = normalize_text(candidate)
        if not q:
            return None

        # 1) exact
        ex = self._pick_best_exact(self.by_norm.get(q, []))
        if ex is not None:
            return self.recipes[ex]

        # 2) contains (unique)
        hits = [i for i, t in enumerate(self.norm_titles) if q in t]
        if len(hits) == 1:
            return self.recipes[hits[0]]

        # 3) optional fuzzy (kapalı tutmak OK)
        if not ENABLE_FUZZY or len(q) < 4:
            return None

        best_idx = None
        best_score = -1
        second_score = -1
        for i, t in enumerate(self.norm_titles):
            s = int(SequenceMatcher(None, q, t).ratio() * 100)
            if s > best_score:
                second_score = best_score
                best_score = s
                best_idx = i
            elif s > second_score:
                second_score = s

        if best_idx is None:
            return None
        if best_score >= FUZZY_THRESHOLD and second_score != best_score:
            return self.recipes[int(best_idx)]
        return None

    def suggest(self, user_text: str, top_n: int = 6, min_score: int = 55) -> List[Recipe]:
        candidate = extract_recipe_name(user_text)
        q = normalize_text(candidate)
        if not q:
            return []

        scored = []
        for i, t in enumerate(self.norm_titles):
            s = int(SequenceMatcher(None, q, t).ratio() * 100)
            scored.append((s, i))
        scored.sort(reverse=True, key=lambda x: x[0])

        out: List[Recipe] = []
        used = set()
        for s, i in scored[: top_n * 6]:
            if s < min_score:
                continue
            r = self.recipes[i]
            if r.norm_title in used:
                continue
            used.add(r.norm_title)
            out.append(r)
            if len(out) >= top_n:
                break
        return out


def format_suggestions(recipes: List[Recipe]) -> str:
    if not recipes:
        return NOT_FOUND_MSG
    lines = [SUGGEST_PREFIX]
    for i, r in enumerate(recipes, 1):
        lines.append(f"{i}) {r.title}")
    lines.append("\nBirini seçmek için numarasını yazabilirsiniz (örn: 1) veya başlığı aynen yazabilirsiniz.")
    return "\n".join(lines).strip()


In [ ]:
# Load authoritative recipes (first run will download+parse; cache used afterwards)
try:
    RECIPES = load_recipes_v3(force_rebuild=False)

    # Eğer cache bozuk/boş geldiyse otomatik rebuild dene
    if not RECIPES or len(RECIPES) == 0:
        print(" Cache empty or parse returned 0 recipes. Rebuilding cache...")
        RECIPES = load_recipes_v3(force_rebuild=True)

    MATCHER = StrictMatcher(RECIPES)

    print(" Loaded recipes:", len(RECIPES))
    print(" Cache path:", CACHE_PATH)

    # Hızlı sanity check (ilk 5 başlık)
    print("\n Sample titles:")
    for r in RECIPES[:5]:
        print(" -", r.title)

except Exception as e:
    print(" Failed to load/parse recipes.")
    print("Error:", repr(e))
    print("\n Tips:")
    print(" - If this is the first run, download can take time.")
    print(" - Try force rebuild: RECIPES = load_recipes_v3(force_rebuild=True)")
    print(" - If using hf:// fallback, ensure datasets version supports it.")
    raise

 Loaded recipes: 3311
 Cache path: /content/recipe_cache/recipes_v3.jsonl

 Sample titles:
 - Sodalı Köfte
 - Hatay Kağıt Kebabı
 - Piliç Topkapı
 - Sebzeli Soslu Tavuk Yemeği
 - Tire Şiş Köfte


In [ ]:
# NumPy 2.x -> 1.x düşür (faiss uyumu için)
!python -m pip -q uninstall -y numpy faiss-cpu
!python -m pip -q install "numpy<2" --no-cache-dir
!python -m pip -q install faiss-cpu==1.8.0 --no-cache-dir

import numpy as np
print(" numpy version:", np.__version__)

import faiss
print(" faiss imported OK")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 63.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtune 0.6.1 requires sentencepiece, which is not installed.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-

In [ ]:
!python -m pip -q install faiss-cpu==1.8.0 sentence-transformers==3.0.1

In [ ]:
import numpy as np
import torch
import faiss
from sentence_transformers import SentenceTransformer

# Embedding modeli (Türkçe için iyi, hızlı)
EMB_MODEL_NAME = "intfloat/multilingual-e5-small"

# RAG ayarları
TOP_K = 5
USE_GPU_FOR_EMB = True  # Colab GPU varsa hızlanır

# 1) Recipe -> tek metin haline getirme (retrieval için)
def recipe_to_doc(r: Recipe) -> str:
    # Retrieval kalitesi için title + category + ingredients + instructions tek metin
    ing = "\n".join([f"- {x}" for x in (r.ingredients or [])])
    steps = "\n".join([f"{i+1}. {x}" for i, x in enumerate(r.instructions or [])])
    return (
        f"Başlık: {r.title}\n"
        f"Kategori: {r.category or ''}\n"
        f"Malzemeler:\n{ing}\n"
        f"Yapılış:\n{steps}\n"
    ).strip()

DOCS = [recipe_to_doc(r) for r in RECIPES]
print("Docs prepared:", len(DOCS))

# 2) Embedding modeli yükle
emb_model = SentenceTransformer(EMB_MODEL_NAME, device=("cuda" if (USE_GPU_FOR_EMB and torch.cuda.is_available()) else "cpu"))

# E5 için query/doc prefix iyi çalışır
DOC_EMB_TEXTS = [("passage: " + d) for d in DOCS]

# 3) Embed + normalize (cosine similarity için)
doc_emb = emb_model.encode(
    DOC_EMB_TEXTS,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

# 4) FAISS index (cosine = inner product + normalized)
dim = doc_emb.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(doc_emb)

print(" FAISS index ready.")
print("Embedding dim:", dim, "| Index size:", index.ntotal)

Docs prepared: 3311


Batches:   0%|          | 0/52 [00:00<?, ?it/s]

 FAISS index ready.
Embedding dim: 384 | Index size: 3311


In [ ]:
import os
import re
import torch

# --- Guard regex'leri ---
_TIME_Q_RE = re.compile(r"(airfryer|air fryer|kaç\s*(dk|dakika|derece)|\b\d+\s*(dk|dakika|°|derece)\b)", re.IGNORECASE)
_PORTION_Q_RE = re.compile(r"(\b\d+\s*kişilik\b|\bporsiyon\b)", re.IGNORECASE)
_SUGGEST_Q_RE = re.compile(r"(\böner\b|\böneri\b|\bne\s*yapayım\b|\bönerir\s*misin\b)", re.IGNORECASE)
_RECIPE_REQ_RE = re.compile(r"(tarif|tarifi|malzeme|yapılış|yapilis|nasıl|nasil)", re.IGNORECASE)

def retrieve_recipes_safe(query: str, top_k: int = 5, score_threshold: float = 0.35, min_margin: float = 0.02):
    """
    - score_threshold: alakasız retrieval'ı keser
    - min_margin: top1 ile top2 çok yakınsa (belirsizlik) keser
    """
    q0 = (query or "").strip()
    if not q0:
        return []

    # Sorudan "tarif adı" gibi kısmı ayıklamaya çalış (yoksa query'nin kendisi)
    q_clean = extract_recipe_name(q0) or q0

    q_emb = emb_model.encode(
        ["query: " + q_clean],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")

    scores, idxs = index.search(q_emb, top_k)
    scores = scores[0].tolist()
    idxs = idxs[0].tolist()

    if not scores or scores[0] < score_threshold:
        return []
    if len(scores) >= 2 and (scores[0] - scores[1]) < min_margin:
        return []

    out = []
    for s, i in zip(scores, idxs):
        if i < 0:
            continue
        out.append((float(s), RECIPES[int(i)]))
    return out


# --------- LLM (Qwen) yükle ----------
from transformers import AutoTokenizer, AutoModelForCausalLM

LLM_NAME = os.environ.get("LLM_NAME", "Qwen/Qwen2.5-3B-Instruct")  # 7B ağırsa 3B daha stabil
tokenizer_llm = AutoTokenizer.from_pretrained(LLM_NAME, use_fast=True, trust_remote_code=True)
model_llm = AutoModelForCausalLM.from_pretrained(
    LLM_NAME,
    torch_dtype=(torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16),
    device_map="auto",
    trust_remote_code=True,
)
model_llm.eval()


LAST_SUGGESTIONS: List[Recipe] = []

def _token_overlap(query: str, title: str) -> int:
    q = normalize_text(extract_recipe_name(query) or query)
    t = normalize_text(title)
    q_tokens = set(q.split())
    t_tokens = set(t.split())
    return len(q_tokens & t_tokens)

def format_suggestions(recipes: List[Recipe]) -> str:
    if not recipes:
        return NOT_FOUND_MSG
    lines = [SUGGEST_PREFIX]
    for i, r in enumerate(recipes, 1):
        lines.append(f"{i}) {r.title}")
    lines.append("\nSeçmek için numara yaz (örn: 1) veya başlığı aynen yaz.")
    return "\n".join(lines).strip()



def rag_answer_guarded(user_question: str, top_k: int = 3) -> str:
    global LAST_SUGGESTIONS

    q = (user_question or "").strip()
    if not q:
        return NOT_FOUND_MSG

    # 0) Kullanıcı numara seçtiyse (1/2/3...)
    if q.isdigit() and LAST_SUGGESTIONS:
        k = int(q) - 1
        if 0 <= k < len(LAST_SUGGESTIONS):
            chosen = LAST_SUGGESTIONS[k]
            LAST_SUGGESTIONS = []
            return format_recipe_block(chosen)

    # 1) Başlık sorularında: önce matcher dene
    best_recipe = None
    if "MATCHER" in globals() and MATCHER is not None:
        exact = MATCHER.match(q)
        if exact is not None:
            LAST_SUGGESTIONS = []
            best_recipe = exact

    # 2) Matcher bulamazsa: FAISS ama "güven" filtresiyle
    hits = []
    if best_recipe is None:
        hits = retrieve_recipes_safe(q, top_k=5, score_threshold=0.35, min_margin=0.02)

        if hits:
            best_score, cand = hits[0]   # (score, recipe)
            overlap = _token_overlap(q, cand.title)

            # >>> anti-sushi kuralı:
            # overlap yoksa çok yüksek skor istemek zorundasın
            ok = (best_score >= 0.45) or (best_score >= 0.38 and overlap >= 1)

            if ok:
                best_recipe = cand
                LAST_SUGGESTIONS = []
            else:
                best_recipe = None

    # 3) Hâlâ recipe yoksa: öneri listesi dön (STRICT: dataset başlıklarından)
    if best_recipe is None:
        sugg = MATCHER.suggest(q, top_n=6) if ("MATCHER" in globals() and MATCHER is not None) else []
        LAST_SUGGESTIONS = sugg[:]  # kullanıcı 1/2/3 seçebilsin
        return format_suggestions(sugg)

    # 4) Guard: süre/derece/airfryer
    if _TIME_Q_RE.search(q):
        return (
            "Bu veri setinde **pişirme süresi / derece / airfryer ayarı** bilgisi yok.\n"
            "Ama ilgili tarifin malzemeleri ve yapılışı şu şekilde:\n\n"
            + format_recipe_block(best_recipe)
        )

    # 5) Guard: porsiyon
    if _PORTION_Q_RE.search(q):
        return (
            "Bu veri setinde **porsiyon/kaç kişilik** bilgisi yok.\n"
            "Ama uygun bir tarif olarak şunu getirebilirim:\n\n"
            + format_recipe_block(best_recipe)
        )

    # 6) “öner” -> LLM yok, direkt tarif
    if _SUGGEST_Q_RE.search(q):
        return "Önerim:\n\n" + format_recipe_block(best_recipe)

    # 7) Tarif/malzeme/yapılış istiyorsa: LLM’e hiç gitme
    if _RECIPE_REQ_RE.search(q) or (len(q.split()) <= 6):
        return format_recipe_block(best_recipe)

    # 8) LLM sadece follow-up için (context sadece kabul edilen recipe/hitler)
    contexts = []
    if hits:
        # hits: [(score, recipe), ...]
        for s, r in hits[:top_k]:
            contexts.append(f"[SCORE={s:.3f}]\n{format_recipe_block(r)}")
    else:
        contexts.append(format_recipe_block(best_recipe))
    context_text = "\n\n---\n\n".join(contexts)

    system = (
        "Sen bir yemek tarifi asistanısın.\n"
        "SADECE verilen CONTEXT içindeki bilgilere dayanarak cevap ver.\n"
        f"Eğer context yeterli değilse veya tarif yoksa aynen şunu yaz: {NOT_FOUND_MSG}\n"
        "Uydurma/ekleme yapma."
    )

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Soru: {q}\n\nCONTEXT:\n{context_text}"},
    ]
    prompt = tokenizer_llm.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer_llm(prompt, return_tensors="pt").to(model_llm.device)

    with torch.inference_mode():
        out = model_llm.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False,
            temperature=0.0,
            repetition_penalty=1.05,
            eos_token_id=tokenizer_llm.eos_token_id,
            pad_token_id=tokenizer_llm.eos_token_id,
        )

    gen_ids = out[0][inputs["input_ids"].shape[1]:]
    text = tokenizer_llm.decode(gen_ids, skip_special_tokens=True).strip()

    for marker in ["<|user|>", "<|assistant|>", "<|system|>"]:
        if marker in text:
            text = text.split(marker, 1)[0].strip()

    if text.startswith(NOT_FOUND_MSG):
        return NOT_FOUND_MSG

    return text

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import gradio as gr

demo_rag = gr.Interface(
    fn=rag_answer_guarded,
    inputs=gr.Textbox(lines=2, label="Soru (örn: 'sodalı köfte tarifi')"),
    outputs=gr.Textbox(lines=25, label="Cevap"),
    title="Türkçe Yemek Tarifi Chatbot (Strict RAG)",
    description="Nefis Yemekler"
  )

demo_rag.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fdcf20e62358f40d00.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
